In [ ]:
import numpy as np
import random
from typing import List, Tuple, Callable

class GeneticAlgorithm:
    def __init__(self, 
                 population_size: int,
                 chromosome_length: int,
                 fitness_function: Callable,
                 mutation_rate: float = 0.01,
                 crossover_rate: float = 0.8):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.fitness_function = fitness_function
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = self._initialize_population()
        
    def _initialize_population(self) -> List[List[int]]:
        """Create initial random population"""
        return [[random.randint(0, 1) for _ in range(self.chromosome_length)] 
                for _ in range(self.population_size)]
    
    def _select_parent(self, fitness_scores: List[float]) -> List[int]:
        """Select parent using roulette wheel selection"""
        total_fitness = sum(fitness_scores)
        selection_probs = [f/total_fitness for f in fitness_scores]
        return self.population[np.random.choice(len(self.population), p=selection_probs)]
    
    def _crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        """Perform single-point crossover"""
        if random.random() < self.crossover_rate:
            point = random.randint(1, self.chromosome_length-1)
            child1 = parent1[:point] + parent2[point:]
            child2 = parent2[:point] + parent1[point:]
            return child1, child2
        return parent1, parent2
    
    def _mutate(self, chromosome: List[int]) -> List[int]:
        """Perform bit-flip mutation"""
        return [1-gene if random.random() < self.mutation_rate else gene 
                for gene in chromosome]
    
    def evolve(self, generations: int) -> Tuple[List[int], float]:
        """Run the genetic algorithm"""
        best_solution = None
        best_fitness = float('-inf')
        
        for generation in range(generations):
            # Evaluate current population
            fitness_scores = [self.fitness_function(chrom) for chrom in self.population]
            
            # Track best solution
            max_fitness_idx = fitness_scores.index(max(fitness_scores))
            if fitness_scores[max_fitness_idx] > best_fitness:
                best_fitness = fitness_scores[max_fitness_idx]
                best_solution = self.population[max_fitness_idx]
            
            # Create new population
            new_population = []
            while len(new_population) < self.population_size:
                # Selection
                parent1 = self._select_parent(fitness_scores)
                parent2 = self._select_parent(fitness_scores)
                
                # Crossover
                child1, child2 = self._crossover(parent1, parent2)
                
                # Mutation
                child1 = self._mutate(child1)
                child2 = self._mutate(child2)
                
                new_population.extend([child1, child2])
            
            # Update population
            self.population = new_population[:self.population_size]
            
            if generation % 10 == 0:
                print(f"Generation {generation}: Best Fitness = {best_fitness}")
        
        return best_solution, best_fitness

# Example usage:
def example_fitness_function(chromosome: List[int]) -> float:
    """Simple fitness function that counts number of 1s"""
    return sum(chromosome)

if __name__ == "__main__":
    # Parameters
    POPULATION_SIZE = 50
    CHROMOSOME_LENGTH = 20
    GENERATIONS = 100
    
    # Create and run GA
    ga = GeneticAlgorithm(
        population_size=POPULATION_SIZE,
        chromosome_length=CHROMOSOME_LENGTH,
        fitness_function=example_fitness_function
    )
    
    best_solution, best_fitness = ga.evolve(GENERATIONS)
    
    print("\nFinal Results:")
    print(f"Best Solution: {best_solution}")
    print(f"Best Fitness: {best_fitness}")

### 1. Travelling Salesman Problem (TSP)

In [ ]:
import numpy as np
from typing import List, Tuple
import random

class TSPSolver():
    def __init__(self, cost_matrix: List[List[int]], population_size=100, generations=500):
        self.population_size = population_size
        self.generations = generations
        self.cost_matrix = np.array(cost_matrix)
        self.cities = len(cost_matrix)
        
    def create_initial_population(self) -> List[List[int]]:
        population = []
        for _ in range(self.population_size):
            chromosome = list(range(1, self.cities))
            random.shuffle(chromosome)
            chromosome = [0] + chromosome
            population.append(chromosome)
        return population
    
    def calculate_fitness(self, chromosome: List[int]) -> float:
        total_distance = 0
        for i in range(self.cities-1):
            total_distance += self.cost_matrix[chromosome[i]][chromosome[i+1]]
        total_distance += self.cost_matrix[chromosome[-1]][chromosome[0]]
        return -total_distance  # Negative because we want to minimize distance
    
    def crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        # Order Crossover (OX)
        point1, point2 = sorted(random.sample(range(1, self.cities), 2))
        
        def create_child(p1, p2):
            child = [-1] * self.cities
            child[0] = 0  # Always start with city 0
            
            # Copy segment from first parent
            child[point1:point2] = p1[point1:point2]
            
            # Fill remaining positions from second parent
            remaining = [x for x in p2 if x not in child[point1:point2] and x != 0]
            for i in range(1, self.cities):
                if child[i] == -1:
                    child[i] = remaining.pop(0)
            
            return child
        
        return create_child(parent1, parent2), create_child(parent2, parent1)
    
    def mutate(self, chromosome: List[int], mutation_rate=0.1) -> List[int]:
        if random.random() < mutation_rate:
            # Swap two random cities (excluding start city)
            i, j = random.sample(range(1, self.cities), 2)
            chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
        return chromosome
    
    def solve(self) -> Tuple[List[int], float]:
        population = self.create_initial_population()
        best_solution = None
        best_fitness = float('-inf')
        
        for generation in range(self.generations):
            # Evaluate fitness
            fitness_scores = [self.calculate_fitness(chrom) for chrom in population]
            
            # Track best solution
            max_fitness_idx = fitness_scores.index(max(fitness_scores))
            if fitness_scores[max_fitness_idx] > best_fitness:
                best_fitness = fitness_scores[max_fitness_idx]
                best_solution = population[max_fitness_idx]
            
            # Selection and reproduction
            new_population = [best_solution]  # Elitism
            
            while len(new_population) < self.population_size:
                # Tournament selection
                tournament_size = 5
                parent1 = max(random.sample(population, tournament_size), 
                            key=lambda x: self.calculate_fitness(x))
                parent2 = max(random.sample(population, tournament_size), 
                            key=lambda x: self.calculate_fitness(x))
                
                # Crossover and mutation
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                new_population.extend([child1, child2])
            
            population = new_population[:self.population_size]
            
            if generation % 50 == 0:
                print(f"Generation {generation}: Best Distance = {-best_fitness}")
        
        return best_solution, -best_fitness



### 2. Optimal Power Flow Problem